In [1]:

from binance.spot import Spot 
from binance.cm_futures import CMFutures
from binance.um_futures import UMFutures
import pandas as pd

cm_futures_client = CMFutures()
um_futures_client = UMFutures()


params = {
    'symbol': 'BTCUSDT',#
    'interval': '1d', #
    'startTime': 1577836800000,# 20 01 01 12 00 am
    # 'endTime': 1513092400000,
    'limit': 1500
    }

response = um_futures_client.klines(**params)
print(type(response[0]))

<class 'list'>


In [2]:
print(response[0])
len(response)

[1577836800000, '7189.43', '7190.52', '7172.94', '7176.26', '1037.337', 1577837699999, '7447960.02135', 1560, '383.751', '2755105.07629', '0']


1500

In [3]:
# Quote asset volume
# Taker buy base asset volume
# Taker buy quote asset volume
df = pd.DataFrame(response, columns =['open_time','open','high', 'low', 'close', 'volume', 'close_time', 'qav', 'num_t', 'tbbav', 'tbqav', 'ignore'])

In [4]:
df.head()

,open_time,open,high,low,close,volume,close_time,qav,num_t,tbbav,tbqav,ignore
0,1577836800000,7189.43,7190.52,7172.94,7176.26,1037.337,1577837699999,7447960.02135,1560,383.751,2755105.07629,0
1,1577837700000,7176.22,7179.41,7170.69,7172.36,707.833,1577838599999,5078345.04373,1199,274.610,1970287.07110,0
2,1577838600000,7172.79,7179.45,7170.61,7174.83,325.246,1577839499999,2333751.74739,447,162.536,1166259.80196,0
3,1577839500000,7174.51,7179.36,7170.15,7171.55,378.633,1577840399999,2716367.62723,482,175.301,1257718.81418,0
4,1577840400000,7171.43,7188.77,7171.10,7186.60,555.389,1577841299999,3987948.79696,807,336.827,2418307.45086,0


In [5]:
print (df.dtypes)

open_time      int64
open          object
high          object
low           object
close         object
volume        object
close_time     int64
qav           object
num_t          int64
tbbav         object
tbqav         object
ignore        object
dtype: object


In [6]:

for col in ['open', 'high','low', 'close', 'volume','qav','tbbav','tbqav','ignore']:
    df[col] = df[col].astype(float)
df.dtypes



open_time       int64
open          float64
high          float64
low           float64
close         float64
volume        float64
close_time      int64
qav           float64
num_t           int64
tbbav         float64
tbqav         float64
ignore        float64
dtype: object

In [7]:
# 현제 시간 timestamp ms 얻기
import time  
ts = time.time()
ts = int(ts*1000)


훈련용 데이터는 일단 1500 개 기본당위로 받고.
그 뒤로는 반복문으로 받을 겁니다. 

-=--
DROP TABLE IF EXISTS public."BTCUSDT_F_15m";
CREATE TABLE IF NOT EXISTS public."BTCUSDT_F_15m"
(
    open_time bigint NOT NULL,
    open numeric,
    high numeric,
    low numeric,
    close numeric,
    volume numeric,
    close_time bigint,
    qav numeric,
    num_t bigint,
    tbbav numeric,
    tbqav numeric,
    ignore numeric,
    CONSTRAINT "BTCUSDT_F_15m_pkey" PRIMARY KEY (open_time)
)

TABLESPACE pg_default;

ALTER TABLE IF EXISTS public."BTCUSDT_F_15m"
    OWNER to postgres;


In [8]:
import psycopg2
import sys

def insert_test(listData):
    try:
        conn = psycopg2.connect(host='localhost', dbname='binance',user='postgres',password='@lstodeks1',port=5432)
        cur = conn.cursor()
        # create table one by one
        # cursor.executemany(sql_str, your_dataframe.values.tolist())
        insert_sql = '''INSERT INTO BTCUSDT_F_15m (open_time, openp, highp, lowp, closep, volume, close_time, qav, num_t, tbbav, tbqav, ign)
          VALUES(%s,%s,%s,%s,%s, %s,%s,%s,%s,%s, %s ,%s)'''
        cur.executemany(insert_sql, listData)                # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
        print('done insert')

    except  (Exception, psycopg2.DatabaseError)  as e:
        print(f'Error {e}')
        sys.exit(1)

    finally:
        if conn:
            conn.close()

insert_test(df.values.tolist())

done insert


In [10]:


def count_test():
    try:
        conn = psycopg2.connect(host='localhost', dbname='binance',user='postgres',password='@lstodeks1',port=5432)
        cur = conn.cursor()
        # create table one by one
        # cursor.executemany(sql_str, your_dataframe.values.tolist())
        insert_sql = '''SELECT count(*) FROM BTCUSDT_F_15m'''
        cur.execute(insert_sql)
        results = cur.fetchone()             # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
        print('done insert')

    except  (Exception, psycopg2.DatabaseError)  as e:
        print(f'Error {e}')
        sys.exit(1)

    finally:
        if conn:
            conn.close()
    return results

print(count_test())

done insert
(1500,)
